# Master Data Catalogue - Duplicate Analysis

This notebook analyzes the Master Data Catalogue indicators CSV file to identify duplicate rows based on indicator title and identifier.

In [40]:
# Import required libraries
import pandas as pd
import numpy as np
from pathlib import Path

# Set display options for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

In [41]:
# Load the Master Data Catalogue indicators CSV file
csv_path = Path("../../hse-data/data/Master Data Catalogue.xlsx - Indicators.csv")

# Check if file exists
if csv_path.exists():
    print(f"✓ File found: {csv_path}")
    # Load the CSV file
    df = pd.read_csv(csv_path)
    print(f"\nDataset shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
else:
    print(f"✗ File not found: {csv_path}")
    print("Available CSV files in data directory:")
    data_dir = Path("../../hse-data/data/")
    for csv_file in data_dir.glob("*.csv"):
        print(f"  - {csv_file.name}")

✓ File found: ../../hse-data/data/Master Data Catalogue.xlsx - Indicators.csv

Dataset shape: (961, 24)
Columns: ['Indicator Title', 'Indicator ID', 'Rationale', 'Definition', 'Indicator Type', 'Status', 'Disaggregations', 'Numerator Data Element', 'Numerator Source Dataset', 'Denominator Data Element', 'Denominator Source Dataset', 'Methodology', 'Report Frequency', 'Report Style', 'Domain', 'Subdomain', 'High-Low Guidance', 'Measurement Limitations', 'Validity Guidance', 'Public Health Importance', 'Provenance', 'Notes', 'Reasons for Inclusion/Exclusion', 'Name of indicator (original source)']


In [42]:
# Examine the data structure
print("=== DATA OVERVIEW ===")
print(f"Shape: {df.shape}")
print(f"\nColumns ({len(df.columns)}):")
for i, col in enumerate(df.columns, 1):
    print(f"{i:2d}. {col}")

print("\n=== FIRST FEW ROWS ===")
display(df.head())

print("\n=== DATA TYPES ===")
print(df.dtypes)

=== DATA OVERVIEW ===
Shape: (961, 24)

Columns (24):
 1. Indicator Title
 2. Indicator ID
 3. Rationale
 4. Definition
 5. Indicator Type
 6. Status
 7. Disaggregations
 8. Numerator Data Element
 9. Numerator Source Dataset
10. Denominator Data Element
11. Denominator Source Dataset
12. Methodology
13. Report Frequency
14. Report Style
15. Domain
16. Subdomain
17. High-Low Guidance
18. Measurement Limitations
19. Validity Guidance
20. Public Health Importance
21. Provenance
22. Notes
23. Reasons for Inclusion/Exclusion
24. Name of indicator (original source)

=== FIRST FEW ROWS ===


,Indicator Title,Indicator ID,Rationale,Definition,Indicator Type,Status,Disaggregations,Numerator Data Element,Numerator Source Dataset,Denominator Data Element,Denominator Source Dataset,Methodology,Report Frequency,Report Style,Domain,Subdomain,High-Low Guidance,Measurement Limitations,Validity Guidance,Public Health Importance,Provenance,Notes,Reasons for Inclusion/Exclusion,Name of indicator (original source)
0,dcterms:title,dct:identifier,hwbp:Rationale,skos:definition,hwbp:indicatorType,hwbp:status,hwbp:disaggregation,hwbp:numeratorDataElement,hwbp:numeratorSource [a dct:dataset [a dct:pub...,hwbp:denominatorDataElement,hwbp:denominatorSource [a dct:dataset [a dct:p...,hwbp:methodology,dcterms:accrualPeriodicity,hwbp:reportStyle,healthdcatap:healthTheme,healthdcatap:healthTheme,hwbp:highLowGuidance,hwbp:measurementLimitations,hwbp:validityGuidance,hwbp:importance,dct:provenance [a dct:ProvenanceStatement;\nrd...,skos:note,skos:historyNote,NaN
1,M,M,R,R,M,R,O,R,M,O,O,R,M,R,R,R,O,O,O,R,O,R,O,NaN
2,Delphi input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Number and proportion of population aged 65 ye...,F1002,NaN,The total number and percentage of people in t...,NaN,Include in Delphi,NaN,NaN,https://data.cso.ie/table/F1002,NaN,NaN,NaN,NaN,NaN,NaN,DEMOGRAPHICS,NaN,NaN,NaN,High,NaN,CSO - Census 2022,NaN,NaN
4,Number of homeless persons aged 65+,NaN,NaN,The number of people aged 65 years and over wh...,NaN,Include in Delphi,NaN,NaN,https://www.gov.ie/en/department-of-housing-lo...,NaN,NaN,NaN,NaN,NaN,NaN,VULNERABLE GROUPS,NaN,NaN,NaN,NaN,NaN,Department of Government and Housing,Publicly available by region for all adults (v...,NaN



=== DATA TYPES ===
Indicator Title                        object
Indicator ID                           object
Rationale                              object
Definition                             object
Indicator Type                         object
Status                                 object
Disaggregations                        object
Numerator Data Element                 object
Numerator Source Dataset               object
Denominator Data Element               object
Denominator Source Dataset             object
Methodology                            object
Report Frequency                       object
Report Style                           object
Domain                                 object
Subdomain                              object
High-Low Guidance                      object
Measurement Limitations                object
Validity Guidance                      object
Public Health Importance               object
Provenance                             object
Notes         

In [43]:
# Find columns that might contain indicator title and identifier
print("=== SEARCHING FOR RELEVANT COLUMNS ===")

# Look for columns containing 'title', 'identifier', 'name', 'id' (case insensitive)
title_columns = [col for col in df.columns if any(keyword in col.lower() for keyword in ['title', 'name'])]
id_columns = [col for col in df.columns if any(keyword in col.lower() for keyword in ['identifier', 'id'])]

print(f"Potential title columns: {title_columns}")
print(f"Potential identifier columns: {id_columns}")

# Display sample values for these columns
if title_columns:
    print(f"\nSample values from '{title_columns[0]}':")
    print(df[title_columns[0]].head(10).tolist())

if id_columns:
    print(f"\nSample values from '{id_columns[0]}':")
    print(df[id_columns[0]].head(10).tolist())

=== SEARCHING FOR RELEVANT COLUMNS ===
Potential title columns: ['Indicator Title', 'Name of indicator (original source)']
Potential identifier columns: ['Indicator ID', 'High-Low Guidance', 'Validity Guidance']

Sample values from 'Indicator Title':
['dcterms:title', 'M', 'Delphi input', 'Number and proportion of population aged 65 years and over', 'Number of homeless persons aged 65+', 'Number of Travellers aged 65+', 'Percentage of older adults (65+) who are in receipt of home care package', 'Deprivation distribution by 8-point Pobal HP index for those aged 65 and over', 'Ethnicity of those aged 65+ ', 'Persons who smoke: Proportion of population aged 65+ who smoked tobacco. (Displayed as: Yes - daily;  Yes - occasionally; No - have given up smoking; Never)']

Sample values from 'Indicator ID':
['dct:identifier', 'M', nan, 'F1002', nan, 'F9026\xa0', 'HU015A_04', nan, 'F9026\xa0', 'F4043']


In [44]:
# Define function to find duplicates based on specified columns
def find_duplicates(dataframe, columns_to_check):
    """
    Find duplicate rows based on specified columns.
    
    Args:
        dataframe: pandas DataFrame
        columns_to_check: list of column names to check for duplicates
    
    Returns:
        tuple: (duplicate_rows, duplicate_summary)
    """
    print(f"=== DUPLICATE ANALYSIS FOR COLUMNS: {columns_to_check} ===")
    
    # Check if columns exist
    missing_cols = [col for col in columns_to_check if col not in dataframe.columns]
    if missing_cols:
        print(f"❌ Missing columns: {missing_cols}")
        return None, None
    
    # Find duplicates
    duplicate_mask = dataframe.duplicated(subset=columns_to_check, keep=False)
    duplicate_rows = dataframe[duplicate_mask].copy()
    
    if len(duplicate_rows) == 0:
        print("✅ No duplicates found!")
        return duplicate_rows, None
    
    # Sort duplicates by the specified columns for better viewing
    duplicate_rows = duplicate_rows.sort_values(by=columns_to_check)
    
    # Create summary
    duplicate_groups = dataframe[duplicate_mask].groupby(columns_to_check).size().reset_index(name='count')
    duplicate_groups = duplicate_groups.sort_values('count', ascending=False)
    
    print(f"📊 SUMMARY:")
    print(f"  Total rows in dataset: {len(dataframe)}")
    print(f"  Duplicate rows found: {len(duplicate_rows)}")
    print(f"  Unique duplicate groups: {len(duplicate_groups)}")
    print(f"  Rows that would remain after deduplication: {len(dataframe) - len(duplicate_rows) + len(duplicate_groups)}")
    
    return duplicate_rows, duplicate_groups

In [45]:
# Define function to find duplicates using OR logic (same title OR same identifier)
def find_duplicates_or_logic(dataframe, title_column, id_column):
    """
    Find duplicate rows where rows are considered duplicates if they have:
    - Same indicator title OR same identifier (not necessarily both)
    
    Args:
        dataframe: pandas DataFrame
        title_column: column name for indicator title
        id_column: column name for indicator identifier
    
    Returns:
        tuple: (duplicate_rows, duplicate_summary)
    """
    print(f"=== DUPLICATE ANALYSIS WITH OR LOGIC ===")
    print(f"Title column: {title_column}")
    print(f"ID column: {id_column}")
    
    # Check if columns exist
    missing_cols = [col for col in [title_column, id_column] if col not in dataframe.columns]
    if missing_cols:
        print(f"❌ Missing columns: {missing_cols}")
        return None, None
    
    # Find duplicates by title (excluding NaN values)
    title_duplicates = dataframe[dataframe.duplicated(subset=[title_column], keep=False) & 
                                dataframe[title_column].notna()]
    
    # Find duplicates by ID (excluding NaN values)
    id_duplicates = dataframe[dataframe.duplicated(subset=[id_column], keep=False) & 
                             dataframe[id_column].notna()]
    
    # Combine both types of duplicates (union)
    all_duplicate_indices = set(title_duplicates.index) | set(id_duplicates.index)
    duplicate_rows = dataframe.loc[list(all_duplicate_indices)].copy()
    
    if len(duplicate_rows) == 0:
        print("✅ No duplicates found!")
        return duplicate_rows, None
    
    # Sort duplicates for better viewing
    duplicate_rows = duplicate_rows.sort_values(by=[title_column, id_column])
    
    # Create detailed summary
    title_groups = title_duplicates.groupby(title_column).size().reset_index(name='count') if len(title_duplicates) > 0 else pd.DataFrame()
    id_groups = id_duplicates.groupby(id_column).size().reset_index(name='count') if len(id_duplicates) > 0 else pd.DataFrame()
    
    print(f"📊 SUMMARY:")
    print(f"  Total rows in dataset: {len(dataframe)}")
    print(f"  Duplicate rows found (OR logic): {len(duplicate_rows)}")
    print(f"  Duplicates by title: {len(title_duplicates)}")
    print(f"  Duplicates by ID: {len(id_duplicates)}")
    print(f"  Unique duplicate groups by title: {len(title_groups)}")
    print(f"  Unique duplicate groups by ID: {len(id_groups)}")
    
    # Create comprehensive summary
    summary_data = []
    
    # Add title-based duplicates
    if len(title_groups) > 0:
        for _, row in title_groups.iterrows():
            summary_data.append({
                'Duplicate_Type': 'Title',
                'Value': row[title_column],
                'Count': row['count'],
                'Column': title_column
            })
    
    # Add ID-based duplicates
    if len(id_groups) > 0:
        for _, row in id_groups.iterrows():
            summary_data.append({
                'Duplicate_Type': 'ID',
                'Value': row[id_column],
                'Count': row['count'],
                'Column': id_column
            })
    
    duplicate_summary = pd.DataFrame(summary_data).sort_values('Count', ascending=False) if summary_data else pd.DataFrame()
    
    return duplicate_rows, duplicate_summary

In [46]:
# Run duplicate analysis with OR logic (same title OR same identifier)
print("=== RUNNING OR LOGIC DUPLICATE ANALYSIS ===\n")

if title_columns and id_columns:
    title_col = title_columns[0]  # 'Indicator Title'
    id_col = id_columns[0]        # 'Indicator ID'
    
    print(f"🎯 Using OR logic with:")
    print(f"  Title column: '{title_col}'")
    print(f"  ID column: '{id_col}'")
    print(f"  Logic: Rows are duplicates if they have the same title OR the same identifier\n")
    
    duplicates_or_df, duplicate_or_summary = find_duplicates_or_logic(df, title_col, id_col)
    
else:
    print("❌ Cannot run OR logic analysis - missing title or ID columns")
    duplicates_or_df = None
    duplicate_or_summary = None

=== RUNNING OR LOGIC DUPLICATE ANALYSIS ===

🎯 Using OR logic with:
  Title column: 'Indicator Title'
  ID column: 'Indicator ID'
  Logic: Rows are duplicates if they have the same title OR the same identifier

=== DUPLICATE ANALYSIS WITH OR LOGIC ===
Title column: Indicator Title
ID column: Indicator ID
📊 SUMMARY:
  Total rows in dataset: 961
  Duplicate rows found (OR logic): 122
  Duplicates by title: 48
  Duplicates by ID: 84
  Unique duplicate groups by title: 24
  Unique duplicate groups by ID: 39


In [53]:
# Display detailed OR logic duplicate analysis results
if 'duplicates_or_df' in locals() and duplicates_or_df is not None:
    print("=== DETAILED OR LOGIC DUPLICATE ANALYSIS ===\n")
    
    if len(duplicates_or_df) > 0:
        print("📋 DUPLICATE GROUPS SUMMARY (OR Logic):")
        if len(duplicate_or_summary) > 0:
            display(duplicate_or_summary)
        
        print(f"\n📄 ALL DUPLICATE ROWS WITH OR LOGIC ({len(duplicates_or_df)} rows):")
        print("Showing all columns for duplicate rows:")
        
        # Temporarily adjust display options for full detail
        with pd.option_context('display.max_colwidth', None, 
                              'display.width', None, 
                              'display.max_columns', None,
                              'display.expand_frame_repr', False):
            display(duplicates_or_df)  # Show all columns with full values
        
        print("\n🔍 DETAILED BREAKDOWN:")
        
        # Show duplicates by title
        title_dups = duplicates_or_df[duplicates_or_df.duplicated(subset=[title_col], keep=False) & 
                                     duplicates_or_df[title_col].notna()]
        if len(title_dups) > 0:
            print(f"\n--- DUPLICATES BY TITLE ({len(title_dups)} rows) ---")
            for title, group_df in title_dups.groupby(title_col):
                print(f"\nTitle: '{title}' - {len(group_df)} duplicates")
                print(f"Row indices: {group_df.index.tolist()}")
                print("Full details for this duplicate group:")
                
                # Show full details for each row in the group
                with pd.option_context('display.max_colwidth', None, 
                                      'display.width', None, 
                                      'display.max_columns', None,
                                      'display.expand_frame_repr', False):
                    display(group_df)
                
                # Additionally show key differences in a more readable format
                print("🔍 Key field differences:")
                for idx, (row_idx, row) in enumerate(group_df.iterrows()):
                    print(f"\n  Row {row_idx} details:")
                    for col in ['Indicator ID', 'Status', 'Definition', 'Disaggregations', 'Domain']:
                        if col in row and pd.notna(row[col]):
                            print(f"    {col}: {row[col]}")
                        elif col in row:
                            print(f"    {col}: [Empty/NaN]")
        
        # Show duplicates by ID
        id_dups = duplicates_or_df[duplicates_or_df.duplicated(subset=[id_col], keep=False) & 
                                  duplicates_or_df[id_col].notna()]
        if len(id_dups) > 0:
            print(f"\n--- DUPLICATES BY ID ({len(id_dups)} rows) ---")
            for id_val, group_df in id_dups.groupby(id_col):
                print(f"\nID: '{id_val}' - {len(group_df)} duplicates")
                print(f"Row indices: {group_df.index.tolist()}")
                print("Full details for this duplicate group:")
                
                # Show full details for each row in the group
                with pd.option_context('display.max_colwidth', None, 
                                      'display.width', None, 
                                      'display.max_columns', None,
                                      'display.expand_frame_repr', False):
                    display(group_df)
                
                # Additionally show key differences in a more readable format
                print("🔍 Key field differences:")
                for idx, (row_idx, row) in enumerate(group_df.iterrows()):
                    print(f"\n  Row {row_idx} details:")
                    for col in ['Indicator Title', 'Status', 'Definition', 'Disaggregations', 'Domain']:
                        if col in row and pd.notna(row[col]):
                            print(f"    {col}: {row[col]}")
                        elif col in row:
                            print(f"    {col}: [Empty/NaN]")
            
    else:
        print("✅ No duplicates found with OR logic - your data is clean!")
        
else:
    print("❌ OR logic duplicate analysis not completed.")

=== DETAILED OR LOGIC DUPLICATE ANALYSIS ===

📋 DUPLICATE GROUPS SUMMARY (OR Logic):


,Duplicate_Type,Value,Count,Column
31,ID,F4055,7,Indicator ID
30,ID,F4014,3,Indicator ID
1,Title,Conversion of ED attendances into admissions,2,Indicator Title
3,Title,How long since a person last smoked - Persons ...,2,Indicator Title
4,Title,How often persons worry they could be a victim...,2,Indicator Title
5,Title,How often persons worry they could be a victim...,2,Indicator Title
0,Title,% posts filled,2,Indicator Title
7,Title,How often persons worry they could be a victim...,2,Indicator Title
8,Title,"Income, Poverty and Deprivation Rates - At Ris...",2,Indicator Title
9,Title,"Income, Poverty and Deprivation Rates - Consis...",2,Indicator Title



📄 ALL DUPLICATE ROWS WITH OR LOGIC (122 rows):
Showing all columns for duplicate rows:


,Indicator Title,Indicator ID,Rationale,Definition,Indicator Type,Status,Disaggregations,Numerator Data Element,Numerator Source Dataset,Denominator Data Element,Denominator Source Dataset,Methodology,Report Frequency,Report Style,Domain,Subdomain,High-Low Guidance,Measurement Limitations,Validity Guidance,Public Health Importance,Provenance,Notes,Reasons for Inclusion/Exclusion,Name of indicator (original source)
745,% posts filled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Integration,NaN,NaN,NaN,NaN,NK,NaN,NaN,NaN,NaN
751,% posts filled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Integration,NaN,NaN,NaN,NaN,NK,NaN,NaN,NaN,NaN
331,Alcohol consumption - Persons who drink more than once per week,HIS15C01,NaN,NaN,NaN,NaN,"Year,Sex,Age Group = ""All ages"",""15 - 19 years"",""20 - 24 years"",""25 - 29 years"",""30 - 34 years"",""35 - 39 years"",""40 - 44 years"",""45 - 49 years"",""50 - 54 years"",""55 - 59 years"",""60 - 64 years"",""65 - 69 years"",""70 - 74 years"",""75 years and over""",Alcohol consumption,https://data.cso.ie/table/HIS15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DOH,"['', 'Department of Health data hosted by the CSO. Any comments or queries please contact Department of Health']",NaN,NaN
394,Antibiotic Consumption,HIS27C01,NaN,NaN,NaN,NaN,"Year,Sex,Age Group = ""All ages"",""15 - 19 years"",""20 - 24 years"",""25 - 29 years"",""30 - 34 years"",""35 - 39 years"",""40 - 44 years"",""45 - 49 years"",""50 - 54 years"",""55 - 59 years"",""60 - 64 years"",""65 - 69 years"",""70 - 74 years"",""75 years and over""",Antibiotic Consumption,https://data.cso.ie/table/HIS27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DOH,"['', 'Department of Health data hosted by the CSO. Any comments or queries can be sent to [url=mailto:info@health.gov.ie]info@health.gov.ie[/url]']",NaN,NaN
100,Consistent Poverty Rate for those over 65+ (%) (Consistent poverty is a broader measure that considers both income poverty and the experience of deprivation (inability to afford basic necessities),SIA61C11,NaN,The percentage of adults aged 65 years and over who live below the poverty line and who lack basic necessities.,NaN,Include in Delphi,NaN,NaN,https://data.cso.ie/table/SIA61,NaN,NaN,NaN,NaN,NaN,NaN,FINANCES,NaN,NaN,NaN,NaN,NaN,CSO - SILC,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,"Provision of care - Yes you do provide care or assistance to one or more persons suffering from some age related problem, chronic health condition or infirmity, at least once a week",IH184C01,NaN,NaN,NaN,NaN,"Year,Age Group = ""15 - 24 years"",""25 - 34 years"",""35 - 44 years"",""45 - 54 years"",""55 - 64 years"",""65 - 74 years"",""75 years and over""",Provision of care,https://data.cso.ie/table/IH184,NaN,NaN,NaN,Every 5 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CSO,"['', 'For more information, please go to the [url=https://www.cso.ie/en/statistics/health/]statistical release page[/url] on our website.\nLevels of deprivation/affluence analysed according to the Pobal HP Index. Please see [url=https://www.cso.ie/en/releasesandpublications/ep/p-ihsc/irishhealthsurvey2019-carersandsocialsupports/backgroundnotes/]background notes[/url] for further information']",NaN,NaN
369,Quality of Life - Persons who rate their quality of life as good or very good,HIS57C01,NaN,NaN,NaN,NaN,"Year,Sex,Age Group = ""All ages"",""15 - 19 years"",""20 - 24 years"",""25 - 29 years"",""30 - 34 years"",""35 - 39 years"",""40 - 44 years"",""45 - 49 years"",""50 - 54 years"",""55 - 59 years"",""60 - 64 years"",""65 - 69 years"",""70 - 74 years"",""75 years and over""",Quality of Life,https://data.cso.ie/table/HIS57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DOH,"['Quality of life is measured by asking respondents to rate their quality of life on a five-point scale from very good to very poor.', 'Department of Health data hosted by the CSO. Any comments or queries can be sent to [url=mailto:info@health.gov.ie]inf


🔍 DETAILED BREAKDOWN:

--- DUPLICATES BY TITLE (48 rows) ---

Title: '% posts filled' - 2 duplicates
Row indices: [745, 751]
Full details for this duplicate group:


...

🔍 Key field differences:

  Row 99 details:
    Indicator Title: Percentage of individuals 65 and above who are feeling lonely by frequency and year
    Status: Include in Delphi
    Definition: The percentage of adults aged 65 years and over who report feeling lonely, shown by year.
    Disaggregations: [Empty/NaN]
    Domain: [Empty/NaN]

  Row 249 details:
    Indicator Title: Percentage of individuals emotional well-being indicators - Individuals feeling lonely
    Status: [Empty/NaN]
    Definition: [Empty/NaN]
    Disaggregations: Year,Age Group = "16 - 24 years","25 - 49 years","50 - 64 years","65 years and over",Time Frequency
    Domain: [Empty/NaN]

ID: 'WBB64C01' - 2 duplicates
Row indices: [153, 109]
Full details for this duplicate group:


,Indicator Title,Indicator ID,Rationale,Definition,Indicator Type,Status,Disaggregations,Numerator Data Element,Numerator Source Dataset,Denominator Data Element,Denominator Source Dataset,Methodology,Report Frequency,Report Style,Domain,Subdomain,High-Low Guidance,Measurement Limitations,Validity Guidance,Public Health Importance,Provenance,Notes,Reasons for Inclusion/Exclusion,Name of indicator (original source)
153,Overall life satisfaction and satisfaction with time use (amount of leisure time) - Individuals overall life satisfaction,WBB64C01,NaN,NaN,NaN,NaN,"Year,Satisfaction Level,Age Group = ""16 - 24 years"",""25 - 49 years"",""50 - 64 years"",""65 years and over""",Overall life satisfaction and satisfaction with time use (amount of leisure time),https://data.cso.ie/table/WBB64,NaN,NaN,NaN,Every 5 years (Annual from 2020),NaN,NaN,NaN,NaN,NaN,NaN,NaN,CSO,"['Scale runs from 0 (Not at all) to 10 (Completely). \r\nSatisfaction with time use only collected in 2022.\r\nThere is a break in the SILC time series for 2020, see [url=https://www.cso.ie/en/releasesandpublications/in/silc/informationnote-breakintimeseriessilc2020/]Information Note[/url].', 'For more information, please go to the [url=https://www.cso.ie/en/statistics/socialconditions/well-being-surveyonincomeandlivingconditionssilc/]statistical release page[/url] on our website.']",NaN,NaN
109,"Percent of people 65 and above with Low, Medium, High, Not Stated overall life satisfaction",WBB64C01,NaN,"The percentage of people aged 65 years and over who satisfied are with their life overall, rated from low to high.",NaN,Include in Delphi,NaN,NaN,https://data.cso.ie/table/WBB64,NaN,NaN,NaN,NaN,NaN,NaN,HEALTH STATUS,NaN,NaN,NaN,NaN,NaN,CSO - SILC Wellbeing,NaN,NaN


🔍 Key field differences:

  Row 153 details:
    Indicator Title: Overall life satisfaction and satisfaction with time use (amount of leisure time) - Individuals overall life satisfaction
    Status: [Empty/NaN]
    Definition: [Empty/NaN]
    Disaggregations: Year,Satisfaction Level,Age Group = "16 - 24 years","25 - 49 years","50 - 64 years","65 years and over"
    Domain: [Empty/NaN]

  Row 109 details:
    Indicator Title: Percent of people 65 and above with Low, Medium, High, Not Stated overall life satisfaction
    Status: Include in Delphi
    Definition: The percentage of people aged 65 years and over who satisfied are with their life overall, rated from low to high.
    Disaggregations: [Empty/NaN]
    Domain: [Empty/NaN]

ID: 'WBD08C01' - 2 duplicates
Row indices: [22, 145]
Full details for this duplicate group:


,Indicator Title,Indicator ID,Rationale,Definition,Indicator Type,Status,Disaggregations,Numerator Data Element,Numerator Source Dataset,Denominator Data Element,Denominator Source Dataset,Methodology,Report Frequency,Report Style,Domain,Subdomain,High-Low Guidance,Measurement Limitations,Validity Guidance,Public Health Importance,Provenance,Notes,Reasons for Inclusion/Exclusion,Name of indicator (original source)
22,Healthy Life Years (HLY) at 65,WBD08C01,NaN,The number of years a person aged 65 years can expect to live in good health without a serious illness or disability.,NaN,Include in Delphi,NaN,NaN,https://data.cso.ie/table/WBD08,NaN,NaN,NaN,NaN,NaN,NaN,HEALTH STATUS,NaN,NaN,NaN,NaN,NaN,CSO - Eurostat (SILC),NaN,NaN
145,Healthy Life Years (HLY) at 65,WBD08C01,NaN,NaN,NaN,NaN,"Year,Sex",Healthy Life Years (HLY) at 65,https://data.cso.ie/table/WBD08,NaN,NaN,NaN,Annual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ESTAT,"['Source: Eurostat\r\nHealthy Life Years is calculated using data on the prevalence of healthy and unhealthy conditions from the Survey on Income and Living Conditions (SILC) and mortality data from Vital Statistics publications. In 2020, the SILC methodology was revised due to the introduction of Regulation 2019/1700 resulting in a break in series. The SILC measure used in the calculation of Healthy Life Years is not considered to be impacted by the break in series, but caution should be applied when comparing data pre and post 2020 for this indicator.', 'For more information, please go to the[url=https://www.cso.ie/en/releasesandpublications/hubs/p-wbhub/well-beinginformationhub/] statistical release page[/url] on our website.']",NaN,NaN


🔍 Key field differences:

  Row 22 details:
    Indicator Title: Healthy Life Years (HLY) at 65
    Status: Include in Delphi
    Definition: The number of years a person aged 65 years can expect to live in good health without a serious illness or disability.
    Disaggregations: [Empty/NaN]
    Domain: [Empty/NaN]

  Row 145 details:
    Indicator Title: Healthy Life Years (HLY) at 65
    Status: [Empty/NaN]
    Definition: [Empty/NaN]
    Disaggregations: Year,Sex
    Domain: [Empty/NaN]


In [48]:
# Save OR logic duplicate results to CSV files
if 'duplicates_or_df' in locals() and duplicates_or_df is not None and len(duplicates_or_df) > 0:
    
    # Create output directory if it doesn't exist
    output_dir = Path("../../hse-data/output")
    output_dir.mkdir(exist_ok=True)
    
    # Save OR logic duplicates to CSV
    or_output_file = output_dir / "master_data_catalogue_duplicates_OR_logic.csv"
    duplicates_or_df.to_csv(or_output_file, index=True)
    
    # Save OR logic summary to CSV
    or_summary_file = output_dir / "master_data_catalogue_duplicate_summary_OR_logic.csv"
    duplicate_or_summary.to_csv(or_summary_file, index=False)
    
    print(f"💾 OR LOGIC RESULTS SAVED:")
    print(f"  Duplicate rows (OR logic): {or_output_file}")
    print(f"  Summary (OR logic): {or_summary_file}")
    
    # Create a clean dataset using OR logic (remove duplicates keeping first occurrence)
    # For OR logic, we need to be more careful about which duplicates to remove
    
    # Remove title duplicates (keep first)
    clean_by_title = df.drop_duplicates(subset=[title_col], keep='first')
    
    # From the remaining data, remove ID duplicates (keep first)
    remaining_indices = clean_by_title.index
    remaining_df = df.loc[remaining_indices]
    clean_by_both = remaining_df.drop_duplicates(subset=[id_col], keep='first')
    
    or_clean_file = output_dir / "master_data_catalogue_cleaned_OR_logic.csv"
    clean_by_both.to_csv(or_clean_file, index=False)
    
    print(f"  Cleaned dataset (OR logic): {or_clean_file}")
    print(f"\n📊 OR LOGIC FINAL SUMMARY:")
    print(f"  Original rows: {len(df)}")
    print(f"  Duplicate rows found: {len(duplicates_or_df)}")
    print(f"  Rows after OR logic deduplication: {len(clean_by_both)}")
    print(f"  Total rows removed: {len(df) - len(clean_by_both)}")
    
    # Comparison with AND logic
    if 'duplicates_df' in locals() and duplicates_df is not None:
        print(f"\n🔄 COMPARISON:")
        print(f"  AND logic duplicates: {len(duplicates_df)} rows")
        print(f"  OR logic duplicates: {len(duplicates_or_df)} rows")
        print(f"  Difference: {len(duplicates_or_df) - len(duplicates_df)} more duplicates found with OR logic")
    
elif 'duplicates_or_df' in locals() and len(duplicates_or_df) == 0:
    print("✅ No duplicates found with OR logic - no files to save!")
    
else:
    print("❌ No OR logic duplicate analysis results to save. Please run the analysis first.")

💾 OR LOGIC RESULTS SAVED:
  Duplicate rows (OR logic): ../../hse-data/output/master_data_catalogue_duplicates_OR_logic.csv
  Summary (OR logic): ../../hse-data/output/master_data_catalogue_duplicate_summary_OR_logic.csv
  Cleaned dataset (OR logic): ../../hse-data/output/master_data_catalogue_cleaned_OR_logic.csv

📊 OR LOGIC FINAL SUMMARY:
  Original rows: 961
  Duplicate rows found: 122
  Rows after OR logic deduplication: 324
  Total rows removed: 637

🔄 COMPARISON:
  AND logic duplicates: 10 rows
  OR logic duplicates: 122 rows
  Difference: 112 more duplicates found with OR logic
  Cleaned dataset (OR logic): ../../hse-data/output/master_data_catalogue_cleaned_OR_logic.csv

📊 OR LOGIC FINAL SUMMARY:
  Original rows: 961
  Duplicate rows found: 122
  Rows after OR logic deduplication: 324
  Total rows removed: 637

🔄 COMPARISON:
  AND logic duplicates: 10 rows
  OR logic duplicates: 122 rows
  Difference: 112 more duplicates found with OR logic


In [49]:
# Run duplicate analysis with automatic column detection
print("=== RUNNING DUPLICATE ANALYSIS ===\n")

# Try to automatically detect the best columns for duplicate checking
if title_columns and id_columns:
    # Use the first title and identifier columns found
    columns_to_check = [title_columns[0], id_columns[0]]
    print(f"🎯 Auto-detected columns for duplicate checking: {columns_to_check}")
    
elif title_columns:
    # Use just the title column
    columns_to_check = [title_columns[0]]
    print(f"🎯 Using title column only: {columns_to_check}")
    
elif id_columns:
    # Use just the identifier column
    columns_to_check = [id_columns[0]]
    print(f"🎯 Using identifier column only: {columns_to_check}")
    
else:
    # Manual specification - you can modify these column names
    print("⚠️  Could not auto-detect columns. Please specify manually:")
    print("Available columns:")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i:2d}. {col}")
    
    # Default attempt with common column names
    possible_columns = ['Indicator Title', 'Indicator Identifier', 'Title', 'Identifier', 'Name', 'ID']
    columns_to_check = [col for col in possible_columns if col in df.columns]
    
    if columns_to_check:
        print(f"\n🔍 Trying with: {columns_to_check}")
    else:
        print("\n❌ Please manually specify the column names in the next cell")
        columns_to_check = None

if columns_to_check:
    duplicates_df, duplicate_summary = find_duplicates(df, columns_to_check)

=== RUNNING DUPLICATE ANALYSIS ===

🎯 Auto-detected columns for duplicate checking: ['Indicator Title', 'Indicator ID']
=== DUPLICATE ANALYSIS FOR COLUMNS: ['Indicator Title', 'Indicator ID'] ===
📊 SUMMARY:
  Total rows in dataset: 961
  Duplicate rows found: 10
  Unique duplicate groups: 2
  Rows that would remain after deduplication: 953


In [50]:
# Display detailed duplicate analysis results
if 'duplicates_df' in locals() and duplicates_df is not None:
    print("=== DETAILED DUPLICATE ANALYSIS ===\n")
    
    if len(duplicates_df) > 0:
        print("📋 DUPLICATE GROUPS SUMMARY:")
        display(duplicate_summary)
        
        print(f"\n📄 ALL DUPLICATE ROWS ({len(duplicates_df)} rows):")
        display(duplicates_df)
        
        print("\n🔍 DUPLICATE GROUPS DETAIL:")
        for group_values, group_df in duplicates_df.groupby(columns_to_check):
            if len(columns_to_check) == 1:
                group_values = [group_values]  # Ensure it's always a list
            
            print(f"\n--- Group: {dict(zip(columns_to_check, group_values))} ---")
            print(f"Number of duplicates: {len(group_df)}")
            print(f"Row indices: {group_df.index.tolist()}")
            
            # Show the actual duplicate rows
            display(group_df[columns_to_check + [col for col in df.columns if col not in columns_to_check][:5]])
            
    else:
        print("✅ No duplicates found - your data is clean!")
        
else:
    print("❌ Duplicate analysis not completed. Please check the column names above.")

=== DETAILED DUPLICATE ANALYSIS ===

📋 DUPLICATE GROUPS SUMMARY:


,Indicator Title,Indicator ID,count
0,Healthy Life Years (HLY) at 65,WBD08C01,2
1,Projected Dependency Ratios from 2022,PEC22C01,2



📄 ALL DUPLICATE ROWS (10 rows):


,Indicator Title,Indicator ID,Rationale,Definition,Indicator Type,Status,Disaggregations,Numerator Data Element,Numerator Source Dataset,Denominator Data Element,Denominator Source Dataset,Methodology,Report Frequency,Report Style,Domain,Subdomain,High-Low Guidance,Measurement Limitations,Validity Guidance,Public Health Importance,Provenance,Notes,Reasons for Inclusion/Exclusion,Name of indicator (original source)
745,% posts filled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Integration,NaN,NaN,NaN,NaN,NK,NaN,NaN,NaN,NaN
751,% posts filled,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Integration,NaN,NaN,NaN,NaN,NK,NaN,NaN,NaN,NaN
91,Conversion of ED attendances into admissions,NaN,NaN,The percentage of Emergency Department visits ...,NaN,Include in Delphi,NaN,NaN,Patient Experience Time,NaN,NaN,NaN,NaN,NaN,NaN,HEALTHCARE USE,NaN,NaN,NaN,NaN,NaN,Patient experience time,https://www.gov.ie/en/collection/f2e25-health-...,NaN
870,Conversion of ED attendances into admissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Admission from ED,NaN,NaN,NaN,NaN,High,NaN,NaN,NaN,NaN
22,Healthy Life Years (HLY) at 65,WBD08C01,NaN,The number of years a person aged 65 years can...,NaN,Include in Delphi,NaN,NaN,https://data.cso.ie/table/WBD08,NaN,NaN,NaN,NaN,NaN,NaN,HEALTH STATUS,NaN,NaN,NaN,NaN,NaN,CSO - Eurostat (SILC),NaN,NaN
145,Healthy Life Years (HLY) at 65,WBD08C01,NaN,NaN,NaN,NaN,"Year,Sex",Healthy Life Years (HLY) at 65,https://data.cso.ie/table/WBD08,NaN,NaN,NaN,Annual,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ESTAT,['Source: Eurostat\r\nHealthy Life Years is ca...,NaN,NaN
44,Number of ED attendances,NaN,NaN,Number of times people aged 65 years and over ...,NaN,Include in Delphi,NaN,NaN,Patient Experience Time,NaN,NaN,NaN,NaN,NaN,NaN,HEALTHCARE USE,NaN,NaN,NaN,NaN,NaN,Patient experience time,https://www.gov.ie/en/collection/f2e25-health-...,NaN
869,Number of ED attendances,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ED attendance,NaN,NaN,NaN,NaN,Medium,NaN,Not broken down by age,NaN,NaN
77,Projected Dependency Ratios from 2022,PEC22C01,NaN,Estimates showing the possible future number o...,NaN,Include in Delphi,NaN,NaN,https://data.cso.ie/table/PEC22,NaN,NaN,NaN,NaN,NaN,NaN,DEMOGRAPHICS,NaN,NaN,NaN,NaN,NaN,CSO - Population and Labour Force Projections,NaN,NaN
144,Projected Dependency Ratios from 2022,PEC22C01,NaN,NaN,NaN,NaN,"Year,Dependents Status,Criteria for Projection",Projected Dependency Ratios from 2022,https://data.cso.ie/table/PEC22,NaN,NaN,NaN,Every 5 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CSO,"['', 'For more information, please go to the [...",NaN,NaN



🔍 DUPLICATE GROUPS DETAIL:

--- Group: {'Indicator Title': 'Healthy Life Years (HLY) at 65', 'Indicator ID': 'WBD08C01'} ---
Number of duplicates: 2
Row indices: [22, 145]


,Indicator Title,Indicator ID,Rationale,Definition,Indicator Type,Status,Disaggregations
22,Healthy Life Years (HLY) at 65,WBD08C01,NaN,The number of years a person aged 65 years can...,NaN,Include in Delphi,NaN
145,Healthy Life Years (HLY) at 65,WBD08C01,NaN,NaN,NaN,NaN,"Year,Sex"



--- Group: {'Indicator Title': 'Projected Dependency Ratios from 2022', 'Indicator ID': 'PEC22C01'} ---
Number of duplicates: 2
Row indices: [77, 144]


,Indicator Title,Indicator ID,Rationale,Definition,Indicator Type,Status,Disaggregations
77,Projected Dependency Ratios from 2022,PEC22C01,NaN,Estimates showing the possible future number o...,NaN,Include in Delphi,NaN
144,Projected Dependency Ratios from 2022,PEC22C01,NaN,NaN,NaN,NaN,"Year,Dependents Status,Criteria for Projection"


In [51]:
# Manual column specification (run this cell if automatic detection failed)
# Uncomment and modify the lines below with the correct column names

# EXAMPLE: If your columns are named differently, specify them here:
# manual_columns = ['Your_Title_Column_Name', 'Your_Identifier_Column_Name']
# duplicates_df, duplicate_summary = find_duplicates(df, manual_columns)

print("💡 TIP: If the automatic detection didn't work, uncomment and modify the lines above")
print("Available columns to choose from:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. '{col}'")

💡 TIP: If the automatic detection didn't work, uncomment and modify the lines above
Available columns to choose from:
   1. 'Indicator Title'
   2. 'Indicator ID'
   3. 'Rationale'
   4. 'Definition'
   5. 'Indicator Type'
   6. 'Status'
   7. 'Disaggregations'
   8. 'Numerator Data Element'
   9. 'Numerator Source Dataset'
  10. 'Denominator Data Element'
  11. 'Denominator Source Dataset'
  12. 'Methodology'
  13. 'Report Frequency'
  14. 'Report Style'
  15. 'Domain'
  16. 'Subdomain'
  17. 'High-Low Guidance'
  18. 'Measurement Limitations'
  19. 'Validity Guidance'
  20. 'Public Health Importance'
  21. 'Provenance'
  22. 'Notes'
  23. 'Reasons for Inclusion/Exclusion'
  24. 'Name of indicator (original source)'


In [52]:
# Save duplicate results to CSV file
if 'duplicates_df' in locals() and duplicates_df is not None and len(duplicates_df) > 0:
    
    # Create output directory if it doesn't exist
    output_dir = Path("../../hse-data/output")
    output_dir.mkdir(exist_ok=True)
    
    # Save duplicates to CSV
    output_file = output_dir / "master_data_catalogue_duplicates.csv"
    duplicates_df.to_csv(output_file, index=True)
    
    # Save summary to CSV
    summary_file = output_dir / "master_data_catalogue_duplicate_summary.csv"
    duplicate_summary.to_csv(summary_file, index=False)
    
    print(f"💾 RESULTS SAVED:")
    print(f"  Duplicate rows: {output_file}")
    print(f"  Summary: {summary_file}")
    
    # Also create a clean dataset (without duplicates)
    clean_df = df.drop_duplicates(subset=columns_to_check, keep='first')
    clean_file = output_dir / "master_data_catalogue_cleaned.csv"
    clean_df.to_csv(clean_file, index=False)
    
    print(f"  Cleaned dataset: {clean_file}")
    print(f"\n📊 FINAL SUMMARY:")
    print(f"  Original rows: {len(df)}")
    print(f"  Duplicate rows removed: {len(df) - len(clean_df)}")
    print(f"  Clean dataset rows: {len(clean_df)}")
    
elif 'duplicates_df' in locals() and len(duplicates_df) == 0:
    print("✅ No duplicates found - no files to save!")
    
else:
    print("❌ No duplicate analysis results to save. Please run the analysis first.")

💾 RESULTS SAVED:
  Duplicate rows: ../../hse-data/output/master_data_catalogue_duplicates.csv
  Summary: ../../hse-data/output/master_data_catalogue_duplicate_summary.csv
  Cleaned dataset: ../../hse-data/output/master_data_catalogue_cleaned.csv

📊 FINAL SUMMARY:
  Original rows: 961
  Duplicate rows removed: 5
  Clean dataset rows: 956
  Cleaned dataset: ../../hse-data/output/master_data_catalogue_cleaned.csv

📊 FINAL SUMMARY:
  Original rows: 961
  Duplicate rows removed: 5
  Clean dataset rows: 956
